In [1]:
import cv2
import mediapipe as mp
from deepface import DeepFace
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_face_detection = mp.solutions.face_detection

# Initialize the face detection model
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Flip the frame horizontally for a more natural selfie-view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform face detection
    results = face_detection.process(frame_rgb)

    # Draw bounding boxes around detected faces
    if results.detections:
        for detection in results.detections:
            mp_drawing.draw_detection(frame, detection)

    # Display the resulting frame
    cv2.imshow('QUALITY CHECK', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()
        #q

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation
mp_face_mesh = mp.solutions.face_mesh

cap = cv2.VideoCapture(0)

with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentation:
    with mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            # Flip the image horizontally for a selfie-view.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

            # Create a segmentation mask.
            segmentation_results = segmentation.process(image)

            # Get the face mesh landmarks.
            face_mesh_results = face_mesh.process(image)

            # Convert the image back to BGR for display.
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw the face mesh landmarks on the image.
            if face_mesh_results.multi_face_landmarks:
                for face_landmarks in face_mesh_results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        face_landmarks,
                        mp_face_mesh.FACEMESH_TESSELATION)

            # Display the image.
            cv2.imshow('MediaPipe Face Mesh', image)

            if cv2.waitKey(5) & 0xFF == 27:
                break

cap.release()
cv2.destroyAllWindows()         #Esc

: 

In [ ]:
img_path = r'C:\Users\vigne\Pictures\Camera Roll\WIN_20240328_20_24_30_Pro.jpg'  # replace with your image path

# Analyze the image
result = DeepFace.analyze(img_path, actions=['emotion'])

# Print the result
print(result)

: 

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

    while True:

        ret, frame = cap.read()

        if not ret:
            break

        image = np.asarray(frame)

        with mp_face_mesh.FaceMesh(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:

            face_mesh_results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if face_mesh_results.multi_face_landmarks:

                for face_landmarks in face_mesh_results.multi_face_landmarks:

                    mp_drawing.draw_landmarks(
                        image,
                        face_landmarks,
                        mp_face_mesh.FACEMESH_TESSELATION)

                result = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)

                emotion = result[0]['dominant_emotion']

                cv2.putText(
                    image,
                    emotion,
                    (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5,
                    (0, 0, 255),
                    3
                )

        # Display the image.
        cv2.imshow('FACE ANALYSIS', image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()    #q